# 1. Install Dependencies and Setup

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [1]:
import os
import cv2
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

# 2. Load Data

### 2.1 Load Images

In [3]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [4]:
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (250,250)))
train_images = train_images.map(lambda x: x/255)

In [5]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (250,250)))
test_images = test_images.map(lambda x: x/255)

In [6]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (250,250)))
val_images = val_images.map(lambda x: x/255)

### 2.2 Prepare Labels

In [7]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
    return [label['keypoints']]

In [8]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [9]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [10]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

### 2.3. Combine Labels and Images

In [11]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(3000)
train = train.batch(30)
train = train.prefetch(4)

In [12]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(300)
test = test.batch(30)
test = test.prefetch(4)

In [13]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(500)
val = val.batch(30)
val = val.prefetch(4)

### 2.4 View Samples

In [14]:
data_samples = train.as_numpy_iterator()

In [15]:
res = data_samples.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][0][idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)

# 3. Build Model

### 3.1 Create Neural Network

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dropout, MaxPooling2D, Flatten, Dense, LeakyReLU
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import VGG16 

In [17]:
model = Sequential([
    Input(shape=(250, 250, 3)),
    VGG16(include_top=False, input_shape=(250, 250, 3)),
    Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='tanh'),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='tanh'),
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='tanh'),
    LeakyReLU(alpha = 0.1),
    Conv2D(filters=8, kernel_size=(2, 2), padding='same', activation='tanh'),
    Dropout(0.1),
    Flatten(),
    Dense(4, activation ='relu')
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 256)         1179904   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 256)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 128)         295040    
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          73792     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 3, 3, 64)          0         
                                                        

### 3.2 Setup Losses and Optimizer

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 256)         1179904   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 256)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 128)         295040    
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          73792     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 3, 3, 64)          0         
                                                        

In [44]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=0.0007)
loss = tf.keras.losses.MeanAbsoluteError()

In [45]:
model.compile(optimizer, loss)

### 3.3 Sense Check predictions

In [39]:
X, y = train.as_numpy_iterator().next()

In [40]:
X.shape

(30, 250, 250, 3)

In [31]:
scores = model.predict(X)

1/1 [==============================] - 3s 3s/step


In [32]:
scores

array([[0.41965175, 0.5693052 , 0.60224265, 0.5284264 ],
       [0.6375619 , 0.6216684 , 0.8106628 , 0.60693204],
       [0.6244768 , 0.5473969 , 0.78598124, 0.5429167 ],
       [0.16599374, 0.7843835 , 0.34710276, 0.7220513 ],
       [0.37860534, 0.62750834, 0.5857409 , 0.5826209 ],
       [0.7548098 , 0.90748245, 0.89694554, 0.90464777],
       [1.0263479 , 0.7474015 , 1.2194008 , 0.7329643 ],
       [1.151741  , 0.7775293 , 1.3476493 , 0.80539405],
       [0.5805922 , 0.70738095, 0.74121374, 0.6733019 ],
       [0.55428773, 0.5646017 , 0.75974154, 0.5515518 ],
       [0.44969213, 0.45002463, 0.66147494, 0.43937445],
       [0.47541723, 0.7011126 , 0.61176836, 0.6313393 ],
       [0.6500641 , 0.55605435, 0.82689977, 0.54724276],
       [0.7308349 , 0.7744889 , 0.9590565 , 0.74891096],
       [0.2991614 , 0.28563762, 0.6218467 , 0.40233415],
       [0.38762343, 0.643011  , 0.5344312 , 0.5860409 ],
       [0.6418172 , 0.5497534 , 0.8031066 , 0.5437675 ],
       [0.6430998 , 0.5645199 ,

### 3.4 Train the Model

In [25]:
# Train for 100 epochs
hist = model.fit(train, epochs=30, validation_data=val)

Epoch 1/30
126/126 [==============================] - 960s 8s/step - loss: 0.0652 - val_loss: 0.0229
Epoch 2/30
126/126 [==============================] - 957s 8s/step - loss: 0.0258 - val_loss: 0.0150
Epoch 3/30
126/126 [==============================] - 955s 8s/step - loss: 0.0191 - val_loss: 0.0176
Epoch 4/30
126/126 [==============================] - 957s 8s/step - loss: 0.0162 - val_loss: 0.0149
Epoch 5/30
126/126 [==============================] - 954s 8s/step - loss: 0.0133 - val_loss: 0.0123
Epoch 6/30
126/126 [==============================] - 952s 8s/step - loss: 0.0116 - val_loss: 0.0126
Epoch 7/30
126/126 [==============================] - 954s 8s/step - loss: 0.0100 - val_loss: 0.0181
Epoch 8/30
126/126 [==============================] - 952s 8s/step - loss: 0.0090 - val_loss: 0.0129
Epoch 9/30
126/126 [==============================] - 952s 8s/step - loss: 0.0082 - val_loss: 0.0129
Epoch 10/30
126/126 [==============================] - 962s 8s/step - loss: 0.0087 - val_lo

KeyboardInterrupt: 

# 4. Review Performance

### 4.1 View Loss Plots

In [41]:
from sklearn.metrics import mean_absolute_error

In [51]:
model = load_model('ResNET6570.h5')

In [52]:
data_samples = test.as_numpy_iterator()

In [53]:
test_sample = test_data.next()

In [55]:
pre = model.predict(test_sample[0])

1/1 [==============================] - 4s 4s/step


In [46]:
mae = model.evaluate(test)

13/13 [==============================] - 37s 3s/step - loss: 0.0209


In [60]:
predicted_labels = np.argmax(pre, axis=1)

In [43]:
loss = model.evaluate(test)

ValueError: in user code:

    File "C:\Users\Nurserik\anaconda3\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\Nurserik\anaconda3\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Nurserik\anaconda3\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\Nurserik\anaconda3\lib\site-packages\keras\engine\training.py", line 1667, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Nurserik\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Nurserik\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 240, in __call__
        self.build(y_pred)
    File "C:\Users\Nurserik\anaco

In [43]:
model = load_model('AlishRes24.h5')

In [33]:
loss = model.evaluate(test)

13/13 [==============================] - 39s 3s/step - loss: 0.0037


In [ ]:
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val loss')
plt.suptitle('Loss')
plt.legend()
plt.show()

### 4.2. Make Predictions on Test

In [26]:
test_data = test.as_numpy_iterator()

In [27]:
test_sample = test_data.next()

In [28]:
yhat = model.predict(test_sample[0])

1/1 [==============================] - 1s 1s/step


In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)

### 4.3. Save the Model

In [26]:
from tensorflow.keras.models import load_model

In [27]:
model.save('VGG5750.h5')

In [ ]:
model = load_model('eyetrackerresnet.h5')

In [28]:
model.predict(test_sample[0])

NameError: name 'test_sample' is not defined

# 5. Real Time Detection

In [48]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    
    frame = frame[50:500,50:500,:] 
    rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb_img, (250,250))
    
    yhat = model.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[0,:4]
    
    cv2.circle(frame, tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(frame, tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 2, (0,255,0), -1)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 173ms/step


1/1 [==============================] - 0s 169ms/step


KeyboardInterrupt: 